In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")


In [3]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)


In [8]:
df = pd.get_dummies(df)

In [9]:
df

,datetime,holiday,workingday,temp,atemp,humidity,windspeed,registered_customer,"weather_clear, few clouds","weather_cloudy, mist",weather_heacy rain or thunderstorm or snow or ice pallets,weather_light snow or rain or thunderstorm
0,2020-01-04 00:00:09,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
1,2020-01-04 00:00:41,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
2,2020-01-04 00:01:20,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
3,2020-01-04 00:04:12,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
4,2020-01-04 00:15:19,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755411,2022-01-03 23:58:28,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755412,2022-01-03 23:58:46,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755413,2022-01-03 23:59:16,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0


In [10]:
df_3 = df.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count","temp": "mean","humidity": "mean", "windspeed": "mean", "weather_clear, few clouds": "mean","weather_cloudy, mist": "mean","weather_heacy rain or thunderstorm or snow or ice pallets": "mean", "weather_light snow or rain or thunderstorm": "mean"}).reset_index()

In [11]:
df_3

,datetime,registered_customer,temp,humidity,windspeed,"weather_clear, few clouds","weather_cloudy, mist",weather_heacy rain or thunderstorm or snow or ice pallets,weather_light snow or rain or thunderstorm
0,2020-01-04 00:00:00,16,9.84,81.0,0.0000,1.0,0.0,0.0,0.0
1,2020-01-04 01:00:00,40,9.02,80.0,0.0000,1.0,0.0,0.0,0.0
2,2020-01-04 02:00:00,32,9.02,80.0,0.0000,1.0,0.0,0.0,0.0
3,2020-01-04 03:00:00,13,9.84,75.0,0.0000,1.0,0.0,0.0,0.0
4,2020-01-04 04:00:00,1,9.84,75.0,0.0000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
17539,2022-01-03 19:00:00,371,10.66,60.0,11.0014,0.0,1.0,0.0,0.0
17540,2022-01-03 20:00:00,282,10.66,60.0,11.0014,0.0,1.0,0.0,0.0
17541,2022-01-03 21:00:00,245,10.66,60.0,11.0014,1.0,0.0,0.0,0.0
17542,2022-01-03 22:00:00,179,10.66,56.0,8.9981,1.0,0.0,0.0,0.0


In [16]:
X = df_3[["temp","datetime","humidity","windspeed","weather_clear, few clouds","weather_cloudy, mist","weather_heacy rain or thunderstorm or snow or ice pallets","weather_light snow or rain or thunderstorm"]]
Y = df_3["registered_customer"]

In [17]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
regr = linear_model.LinearRegression()
regr.fit(X,Y)
# print("inputs", X.columns,"\n")
# print("Corresponding: \n", regr.coef_)
# print("Intercept: \n", regr.intercept_)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[float64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>)